<a href="https://colab.research.google.com/github/purvadanait/Final-Year-Project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
!pip install fastapi
!pip install python-utils
!pip install python-multipart

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=51074205b3ada317fc169c869e6cbb7571f6570de968185efa553a0b9632a21a
  Stored in directory: /root/.cache/pip/wheels/9e/fc/1c/cf980e6413d3ee8e70cd8f39e2366b0f487e3e221aeb452eb0
Successfully built python-multipart


In [46]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [47]:
!ls "/content/drive/My Drive/inference_utils.py"

'/content/drive/My Drive/inference_utils.py'


In [48]:
!cat "/content/drive/My Drive/inference_utils.py"

# -*- coding: utf-8 -*-
"""inference_utils.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1-E0roVmceQ6ODYmZ6E7nw0D7pigppn0W
"""

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
sys.path.append("/content/drive/My Drive")

import cv2_utils

import sys
sys.path.append("/content/drive/My Drive")

import deep_crack

import os
import cv2
import torch
import numpy as np
from PIL import Image
from io import BytesIO
from cv2_utils import getContours
import torchvision.transforms as transforms
from deep_crack import DeepCrack

def tensor2im(input_image, imtype=np.uint8):
    """"Converts a Tensor array into a numpy image array.
    Parameters:
        input_image (tensor) --  the input image tensor array
        imtype (type)        --  the desired type of the converted numpy array
    """
    if not isinstance(input_image, np.ndarray):
        if isinstance(input_image, torch.T

In [49]:
import sys
sys.path.append("/content/drive/My Drive")

In [50]:
import inference_utils

Mounted at /content/drive


In [52]:
from typing import Union
from fastapi import FastAPI
from pydantic import BaseModel
from inference_utils import inference, create_model
from typing import List, Optional
from pydantic import BaseModel
from fastapi import UploadFile, File
import io
from starlette.responses import StreamingResponse
from fastapi.middleware.cors import CORSMiddleware
import numpy as np
import cv2
from PIL import Image
import base64


In [53]:
class Parameters(BaseModel):
    input_nc: int = 3
    output_nc: int = 3
    ngf: int = 64
    ndf: int = 64
    netG:str = 'resnet_9blocks'
    norm: str = 'instance'
    init_type: str = 'xavier'
    init_gain: float = 0.02
    display_sides: int = 1
    num_classes: int = 1
    gpu_ids: List[int] = [0]
    isTrain: bool = False

In [54]:
app = FastAPI()
origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [55]:
@app.get("/")
def read_root():
    return {"Hello": "World"}

def from_image_to_bytes(img):
    """
    pillow image to bytes
    """
    imgByteArr = io.BytesIO()
    img.save(imgByteArr, format='png')
    imgByteArr = imgByteArr.getvalue()

    encoded = base64.b64encode(imgByteArr)
    decoded = encoded.decode('ascii')
    return decoded

In [58]:
@app.post("/predict/{dim}/{unit}")
async def predict(dim:str, unit:str, file: UploadFile = File(...)):
    bytesImg = await file.read()
    width, height = [float(x) for x in dim.split('-')]    
    parameters = Parameters()
    model = create_model(parameters)
    predicted_cntr, visuals = inference(model, bytesImg, (width, height), unit) # Outputs Predicted Masks

    cntr_converted = from_image_to_bytes(Image.fromarray(predicted_cntr))
    
    img_list = [cntr_converted]
    
    for k in ['fused', 'side1', 'side2', 'side3', 'side4', 'side5']:
        map_ = from_image_to_bytes(Image.fromarray(visuals[k]))
        img_list.append(map_)
    return img_list
